# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2022 1:45PM,239476,21,617686,"NBTY Global, Inc.",Released
1,Jun 13 2022 1:45PM,239475,21,617688,"NBTY Global, Inc.",Released
2,Jun 13 2022 1:43PM,239474,21,399280395,"NBTY Global, Inc.",Released
3,Jun 13 2022 1:41PM,239473,19,TR-RN-06152022,"GCH Granules USA, Inc.",Released
4,Jun 13 2022 1:41PM,239472,19,ADV80003286,"AdvaGen Pharma, Ltd",Released
5,Jun 13 2022 1:41PM,239472,19,ADV80003287,"AdvaGen Pharma, Ltd",Released
6,Jun 13 2022 1:41PM,239472,19,ADV80003288,"AdvaGen Pharma, Ltd",Released
7,Jun 13 2022 1:41PM,239472,19,ADV80003289,"AdvaGen Pharma, Ltd",Released
8,Jun 13 2022 1:41PM,239472,19,ADV80003290,"AdvaGen Pharma, Ltd",Released
9,Jun 13 2022 1:41PM,239472,19,ADV80003291,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,239472,Released,20
36,239473,Released,1
37,239474,Released,1
38,239475,Released,1
39,239476,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239472,NaN,NaN,20.0
239473,NaN,NaN,1.0
239474,NaN,NaN,1.0
239475,NaN,NaN,1.0
239476,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239388,0.0,0.0,1.0
239389,0.0,0.0,1.0
239392,0.0,0.0,1.0
239396,0.0,0.0,1.0
239398,0.0,0.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239388,0,0,1
239389,0,0,1
239392,0,0,1
239396,0,0,1
239398,0,0,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239388,0,0,1
1,239389,0,0,1
2,239392,0,0,1
3,239396,0,0,1
4,239398,0,0,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239388,,,1
1,239389,,,1
2,239392,,,1
3,239396,,,1
4,239398,,,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2022 1:45PM,239476,21,"NBTY Global, Inc."
1,Jun 13 2022 1:45PM,239475,21,"NBTY Global, Inc."
2,Jun 13 2022 1:43PM,239474,21,"NBTY Global, Inc."
3,Jun 13 2022 1:41PM,239473,19,"GCH Granules USA, Inc."
4,Jun 13 2022 1:41PM,239472,19,"AdvaGen Pharma, Ltd"
24,Jun 13 2022 1:32PM,239471,20,"Exact-Rx, Inc."
46,Jun 13 2022 1:19PM,239470,10,"Snap Medical Industries, LLC"
58,Jun 13 2022 12:46PM,239463,20,"ACI Healthcare USA, Inc."
87,Jun 13 2022 12:36PM,239467,21,"NBTY Global, Inc."
88,Jun 13 2022 12:13PM,239465,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2022 1:45PM,239476,21,"NBTY Global, Inc.",,,1
1,Jun 13 2022 1:45PM,239475,21,"NBTY Global, Inc.",,,1
2,Jun 13 2022 1:43PM,239474,21,"NBTY Global, Inc.",,,1
3,Jun 13 2022 1:41PM,239473,19,"GCH Granules USA, Inc.",,,1
4,Jun 13 2022 1:41PM,239472,19,"AdvaGen Pharma, Ltd",,,20
5,Jun 13 2022 1:32PM,239471,20,"Exact-Rx, Inc.",,,22
6,Jun 13 2022 1:19PM,239470,10,"Snap Medical Industries, LLC",,,12
7,Jun 13 2022 12:46PM,239463,20,"ACI Healthcare USA, Inc.",,23,6
8,Jun 13 2022 12:36PM,239467,21,"NBTY Global, Inc.",,,1
9,Jun 13 2022 12:13PM,239465,20,Alumier Labs Inc.,,,29


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 1:45PM,239476,21,"NBTY Global, Inc.",1,,
1,Jun 13 2022 1:45PM,239475,21,"NBTY Global, Inc.",1,,
2,Jun 13 2022 1:43PM,239474,21,"NBTY Global, Inc.",1,,
3,Jun 13 2022 1:41PM,239473,19,"GCH Granules USA, Inc.",1,,
4,Jun 13 2022 1:41PM,239472,19,"AdvaGen Pharma, Ltd",20,,
5,Jun 13 2022 1:32PM,239471,20,"Exact-Rx, Inc.",22,,
6,Jun 13 2022 1:19PM,239470,10,"Snap Medical Industries, LLC",12,,
7,Jun 13 2022 12:46PM,239463,20,"ACI Healthcare USA, Inc.",6,23,
8,Jun 13 2022 12:36PM,239467,21,"NBTY Global, Inc.",1,,
9,Jun 13 2022 12:13PM,239465,20,Alumier Labs Inc.,29,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 1:45PM,239476,21,"NBTY Global, Inc.",1,,
1,Jun 13 2022 1:45PM,239475,21,"NBTY Global, Inc.",1,,
2,Jun 13 2022 1:43PM,239474,21,"NBTY Global, Inc.",1,,
3,Jun 13 2022 1:41PM,239473,19,"GCH Granules USA, Inc.",1,,
4,Jun 13 2022 1:41PM,239472,19,"AdvaGen Pharma, Ltd",20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 1:45PM,239476,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 13 2022 1:45PM,239475,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jun 13 2022 1:43PM,239474,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 13 2022 1:41PM,239473,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Jun 13 2022 1:41PM,239472,19,"AdvaGen Pharma, Ltd",20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 1:45PM,239476,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 13 2022 1:45PM,239475,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jun 13 2022 1:43PM,239474,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 13 2022 1:41PM,239473,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Jun 13 2022 1:41PM,239472,19,"AdvaGen Pharma, Ltd",20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1676118,190.0,1.0,0.0
12,478912,0.0,2.0,0.0
15,1197211,48.0,41.0,19.0
16,239407,0.0,1.0,0.0
18,1436432,6.0,0.0,0.0
19,718379,22.0,0.0,0.0
20,957797,71.0,23.0,0.0
21,1197356,5.0,0.0,0.0
22,478777,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1676118,190.0,1.0,0.0
1,12,478912,0.0,2.0,0.0
2,15,1197211,48.0,41.0,19.0
3,16,239407,0.0,1.0,0.0
4,18,1436432,6.0,0.0,0.0
5,19,718379,22.0,0.0,0.0
6,20,957797,71.0,23.0,0.0
7,21,1197356,5.0,0.0,0.0
8,22,478777,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,190.0,1.0,0.0
1,12,0.0,2.0,0.0
2,15,48.0,41.0,19.0
3,16,0.0,1.0,0.0
4,18,6.0,0.0,0.0
5,19,22.0,0.0,0.0
6,20,71.0,23.0,0.0
7,21,5.0,0.0,0.0
8,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,190.0
1,12,Released,0.0
2,15,Released,48.0
3,16,Released,0.0
4,18,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,1.0,2.0,41.0,1.0,0.0,0.0,23.0,0.0,0.0
Released,190.0,0.0,48.0,0.0,6.0,22.0,71.0,5.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,2.0,41.0,1.0,0.0,0.0,23.0,0.0,0.0
2,Released,190.0,0.0,48.0,0.0,6.0,22.0,71.0,5.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,2.0,41.0,1.0,0.0,0.0,23.0,0.0,0.0
2,Released,190.0,0.0,48.0,0.0,6.0,22.0,71.0,5.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()